# Azure AI Foundry Application Tracing

This notebook demonstrates how to implement tracing for AI applications using the OpenAI SDK with OpenTelemetry in Azure AI Foundry. Tracing provides deep visibility into execution of your application by capturing detailed telemetry at each execution step.

## Prerequisites
- An Azure AI Foundry project created
- An AI application that uses OpenAI SDK to make calls to models hosted in Azure AI Foundry
- Application Insights resource configured for your Azure AI Foundry project

## 1. Install Required Packages

Install the Azure AI Foundry SDK and other required packages for application tracing.

## 2. Import Libraries and Initialize Client

Import necessary libraries and initialize the Azure AI Foundry client with proper authentication.

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter

print("Libraries imported successfully!")

In [ ]:
# Configure your Azure AI Foundry project details
# Replace these with your actual values
from dotenv import load_dotenv
load_dotenv()
AZURE_AI_PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
# Alternative: You can also use environment variables
# AZURE_AI_PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
# Initialize the AI Project Client
try:
    project_client = AIProjectClient(
        credential=DefaultAzureCredential(),
        endpoint=AZURE_AI_PROJECT_ENDPOINT,
    )
    print("AI Project Client initialized successfully!")
except Exception as e:
    print(f"Error initializing client: {e}")
    print("Please ensure your Azure credentials are configured and the endpoint is correct.")

## 3. Configure Tracing Settings

Set up tracing configuration including project settings and trace collection parameters.

In [ ]:
# Step 1: Instrument the OpenAI SDK
OpenAIInstrumentor().instrument()
print("OpenAI SDK instrumented for tracing!")

# Step 2: Configure environment variable to capture inputs and outputs
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"
print("Message content capture enabled!")

In [ ]:
# Get the connection string to Azure Application Insights
try:
    connection_string = project_client.telemetry.get_application_insights_connection_string()
    print(f"Connection string retrieved: {connection_string[:50]}...")
    
    # Configure Azure Monitor for telemetry
    configure_azure_monitor(connection_string=connection_string)
    print("Azure Monitor configured successfully!")
    
except Exception as e:
    print(f"Error getting connection string: {e}")
    print("Make sure Application Insights is configured for your Azure AI Foundry project.")
    
    # Fallback: Configure console tracing for testing
    print("\nConfiguring console tracing as fallback...")
    span_exporter = ConsoleSpanExporter()
    tracer_provider = TracerProvider()
    tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter))
    trace.set_tracer_provider(tracer_provider)
    print("Console tracing configured!")

## 4. Create a Simple Traced Function

Implement a basic function with tracing decorators to demonstrate fundamental tracing concepts.

In [ ]:
# Get the OpenAI client from the project
try:
    client = project_client.get_openai_client()
    print("OpenAI client retrieved successfully!")
except Exception as e:
    print(f"Error getting OpenAI client: {e}")
    # For testing purposes, you might want to create a client directly
    # from openai import OpenAI
    # client = OpenAI(api_key="your-api-key", base_url="your-base-url")

In [ ]:
# Create a tracer instance
tracer = trace.get_tracer(__name__)

# Simple traced function
@tracer.start_as_current_span("simple_chat_completion")
def simple_chat_completion(message: str, model: str = deployment_name):
    """
    A simple function that makes a chat completion request with tracing.
    """
    current_span = trace.get_current_span()
    
    # Add custom attributes to the span
    current_span.set_attribute("user.message_length", len(message))
    current_span.set_attribute("user.model_requested", model)
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": message}
            ],
            max_tokens=200
        )
        
        # Add response attributes
        current_span.set_attribute("response.completion_tokens", response.usage.completion_tokens)
        current_span.set_attribute("response.prompt_tokens", response.usage.prompt_tokens)
        current_span.set_attribute("response.total_tokens", response.usage.total_tokens)
        
        return response.choices[0].message.content
        
    except Exception as e:
        current_span.record_exception(e)
        current_span.set_status(trace.Status(trace.StatusCode.ERROR, str(e)))
        raise

print("Simple traced function created!")

In [ ]:
# Test the simple traced function
try:
    result = simple_chat_completion("Write a short poem about tracing in AI applications.")
    print("Response received:")
    print(result)
except Exception as e:
    print(f"Error in simple chat completion: {e}")

## 5. Trace a Multi-Step Workflow

Build a more complex workflow with multiple steps and nested function calls to show hierarchical tracing.

In [ ]:
def build_prompt_with_context(claim: str, context: str) -> list:
    """
    Build a prompt for assessing claims based on provided context.
    """
    return [
        {
            'role': 'system', 
            'content': "I will ask you to assess whether a particular scientific claim is supported by the evidence provided. Output only 'True' if the claim is true, 'False' if the claim is false, or 'NEE' if there's not enough evidence."
        },
        {
            'role': 'user', 
            'content': f"""
                The evidence is the following: {context}

                Assess the following claim on the basis of the evidence. Output only 'True' if the claim is true, 'False' if the claim is false, or 'NEE' if there's not enough evidence. Do not output any other text.

                Claim:
                {claim}

                Assessment:
            """
        }
    ]

@tracer.start_as_current_span("assess_single_claim")
def assess_single_claim(claim: str, context: str, model: str = "gpt-4"):
    """
    Assess a single claim against provided context.
    """
    current_span = trace.get_current_span()
    current_span.set_attribute("claim.length", len(claim))
    current_span.set_attribute("context.length", len(context))
    
    messages = build_prompt_with_context(claim=claim, context=context)
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=10
    )
    
    result = response.choices[0].message.content.strip('., ')
    current_span.set_attribute("assessment.result", result)
    
    return result

@tracer.start_as_current_span("assess_claims_with_context")
def assess_claims_with_context(claims: list, contexts: list, model: str = deployment_name):
    """
    Assess multiple claims against their respective contexts.
    """
    current_span = trace.get_current_span()
    current_span.set_attribute("operation.claims_count", len(claims))
    current_span.set_attribute("operation.model", model)
    
    responses = []
    
    for i, (claim, context) in enumerate(zip(claims, contexts)):
        # Create a child span for each individual assessment
        with tracer.start_as_current_span(f"assess_claim_{i+1}"):
            result = assess_single_claim(claim, context, model)
            responses.append(result)
    
    current_span.set_attribute("operation.completed_assessments", len(responses))
    return responses

print("Multi-step workflow functions created!")

In [ ]:
# Test the multi-step workflow
sample_claims = [
    "Python is a programming language",
    "The Earth is flat",
    "Machine learning requires large datasets"
]

sample_contexts = [
    "Python is a high-level, interpreted programming language with dynamic semantics. It was created by Guido van Rossum and first released in 1991.",
    "The Earth is an oblate spheroid, meaning it's mostly spherical but slightly flattened at the poles due to its rotation. This has been confirmed by satellite imagery and space exploration.",
    "While machine learning can benefit from large datasets, many techniques work effectively with smaller datasets. Transfer learning, few-shot learning, and data augmentation are examples of approaches that work with limited data."
]

try:
    assessments = assess_claims_with_context(sample_claims, sample_contexts)
    print("Assessment Results:")
    for claim, assessment in zip(sample_claims, assessments):
        print(f"Claim: {claim[:50]}...")
        print(f"Assessment: {assessment}\n")
except Exception as e:
    print(f"Error in multi-step workflow: {e}")

## 6. Add Custom Attributes and Events

Enhance traces with custom attributes, tags, and events to provide richer debugging information.

In [ ]:
import time
import json
from datetime import datetime

@tracer.start_as_current_span("enhanced_chat_with_attributes")
def enhanced_chat_with_attributes(message: str, model: str = deployment_name, temperature: float = 0.7):
    """
    Enhanced chat function with rich tracing attributes and events.
    """
    current_span = trace.get_current_span()
    start_time = time.time()
    
    # Set initial attributes
    current_span.set_attribute("user.message", message[:100])  # Truncate for privacy
    current_span.set_attribute("model.name", model)
    current_span.set_attribute("model.temperature", temperature)
    current_span.set_attribute("session.timestamp", datetime.now().isoformat())
    current_span.set_attribute("session.user_id", "demo_user")
    
    # Add an event for the start of processing
    current_span.add_event(
        "processing_started",
        {
            "message_length": len(message),
            "processing_timestamp": datetime.now().isoformat()
        }
    )
    
    try:
        # Add event before API call
        current_span.add_event("api_call_initiated")
        
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}],
            temperature=temperature,
            max_tokens=300
        )
        
        # Calculate processing time
        processing_time = time.time() - start_time
        
        # Add success attributes
        current_span.set_attribute("response.success", True)
        current_span.set_attribute("response.processing_time_seconds", processing_time)
        current_span.set_attribute("response.completion_tokens", response.usage.completion_tokens)
        current_span.set_attribute("response.prompt_tokens", response.usage.prompt_tokens)
        current_span.set_attribute("response.total_tokens", response.usage.total_tokens)
        current_span.set_attribute("response.model_used", response.model)
        
        # Add success event
        current_span.add_event(
            "response_received",
            {
                "response_length": len(response.choices[0].message.content),
                "finish_reason": response.choices[0].finish_reason,
                "processing_time": processing_time
            }
        )
        
        # Add performance warning if response took too long
        if processing_time > 5.0:
            current_span.add_event(
                "performance_warning",
                {"message": "Response time exceeded 5 seconds", "actual_time": processing_time}
            )
        
        return {
            "content": response.choices[0].message.content,
            "processing_time": processing_time,
            "token_usage": {
                "prompt_tokens": response.usage.prompt_tokens,
                "completion_tokens": response.usage.completion_tokens,
                "total_tokens": response.usage.total_tokens
            }
        }
        
    except Exception as e:
        # Add error attributes and events
        current_span.set_attribute("response.success", False)
        current_span.set_attribute("error.type", type(e).__name__)
        current_span.set_attribute("error.message", str(e))
        
        current_span.add_event(
            "error_occurred",
            {
                "error_type": type(e).__name__,
                "error_message": str(e),
                "processing_time_at_error": time.time() - start_time
            }
        )
        
        # Record the exception in the span
        current_span.record_exception(e)
        current_span.set_status(trace.Status(trace.StatusCode.ERROR, str(e)))
        
        raise

print("Enhanced function with custom attributes and events created!")

In [ ]:
# Test the enhanced function
try:
    result = enhanced_chat_with_attributes(
        "Explain the benefits of application tracing in AI systems.",
        temperature=0.7
    )
    
    print("Enhanced Response:")
    print(result["content"])
    print(f"\nProcessing time: {result['processing_time']:.2f} seconds")
    print(f"Token usage: {result['token_usage']}")
    
except Exception as e:
    print(f"Error in enhanced chat: {e}")

## 7. Trace LLM Calls

Implement tracing for Large Language Model calls, including input/output logging and performance metrics.

In [ ]:
@tracer.start_as_current_span("llm_conversation_chain")
def llm_conversation_chain(initial_prompt: str, follow_up_questions: list, model: str = deployment_name):
    """
    Simulate a conversation chain with multiple LLM calls and comprehensive tracing.
    """
    current_span = trace.get_current_span()
    current_span.set_attribute("conversation.initial_prompt", initial_prompt[:200])
    current_span.set_attribute("conversation.follow_up_count", len(follow_up_questions))
    current_span.set_attribute("conversation.model", model)
    
    conversation_history = []
    total_tokens = 0
    
    # Initial call
    with tracer.start_as_current_span("initial_llm_call") as initial_span:
        initial_span.set_attribute("call.type", "initial")
        initial_span.set_attribute("call.prompt", initial_prompt[:200])
        
        messages = [{"role": "user", "content": initial_prompt}]
        
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=200
        )
        
        initial_response = response.choices[0].message.content
        conversation_history.append({"role": "user", "content": initial_prompt})
        conversation_history.append({"role": "assistant", "content": initial_response})
        
        initial_tokens = response.usage.total_tokens
        total_tokens += initial_tokens
        
        initial_span.set_attribute("call.tokens_used", initial_tokens)
        initial_span.set_attribute("call.response_length", len(initial_response))
    
    # Follow-up calls
    for i, follow_up in enumerate(follow_up_questions):
        with tracer.start_as_current_span(f"follow_up_call_{i+1}") as follow_up_span:
            follow_up_span.set_attribute("call.type", "follow_up")
            follow_up_span.set_attribute("call.index", i + 1)
            follow_up_span.set_attribute("call.question", follow_up[:200])
            follow_up_span.set_attribute("call.conversation_length", len(conversation_history))
            
            # Add the follow-up question to conversation
            conversation_history.append({"role": "user", "content": follow_up})
            
            response = client.chat.completions.create(
                model=model,
                messages=conversation_history,
                max_tokens=200
            )
            
            follow_up_response = response.choices[0].message.content
            conversation_history.append({"role": "assistant", "content": follow_up_response})
            
            follow_up_tokens = response.usage.total_tokens
            total_tokens += follow_up_tokens
            
            follow_up_span.set_attribute("call.tokens_used", follow_up_tokens)
            follow_up_span.set_attribute("call.response_length", len(follow_up_response))
            follow_up_span.set_attribute("call.cumulative_tokens", total_tokens)
    
    # Final span attributes
    current_span.set_attribute("conversation.total_tokens", total_tokens)
    current_span.set_attribute("conversation.total_exchanges", len(conversation_history) // 2)
    current_span.set_attribute("conversation.final_length", len(conversation_history))
    
    # Add final event
    current_span.add_event(
        "conversation_completed",
        {
            "total_exchanges": len(conversation_history) // 2,
            "total_tokens": total_tokens,
            "conversation_length": len(conversation_history)
        }
    )
    
    return {
        "conversation": conversation_history,
        "total_tokens": total_tokens,
        "exchanges": len(conversation_history) // 2
    }

print("LLM conversation chain function created!")

In [ ]:
# Test the LLM conversation chain
try:
    result = llm_conversation_chain(
        initial_prompt="What is machine learning?",
        follow_up_questions=[
            "What are the main types of machine learning?",
            "Give me an example of supervised learning."
        ]
    )
    
    print("Conversation Chain Results:")
    print(f"Total exchanges: {result['exchanges']}")
    print(f"Total tokens used: {result['total_tokens']}")
    print("\nConversation:")
    
    for i, message in enumerate(result['conversation']):
        role = message['role'].capitalize()
        content = message['content'][:100] + "..." if len(message['content']) > 100 else message['content']
        print(f"{i+1}. {role}: {content}\n")
        
except Exception as e:
    print(f"Error in conversation chain: {e}")

## 8. View and Analyze Traces

Access and examine collected traces through the Azure AI Foundry portal and programmatically.

In [ ]:
# Information about viewing traces
print("""Viewing and Analyzing Traces:

1. Azure AI Foundry Portal:
   - Go to https://ai.azure.com
   - Navigate to your project
   - Click on "Tracing" in the side navigation
   - View traces in real-time or filter by time range

2. Application Insights:
   - Access your Application Insights resource in Azure Portal
   - Use "Transaction search" to find specific traces
   - Use "Application map" to see component interactions
   - Create custom queries using KQL (Kusto Query Language)

3. Programmatic Access:
   - Use Azure Monitor Query API
   - Export traces for analysis
   - Create dashboards and alerts

Key Trace Information Available:
- Execution time and performance metrics
- Input/output data (when enabled)
- Error details and stack traces
- Custom attributes and events
- Token usage and costs
- Request/response patterns
""")

# Display current project endpoint for reference
if 'AZURE_AI_PROJECT_ENDPOINT' in locals():
    print(f"\nYour project endpoint: {AZURE_AI_PROJECT_ENDPOINT}")
    print("Visit the tracing section in Azure AI Foundry to see your traces!")

In [ ]:
# Example of a function that helps with trace analysis
@tracer.start_as_current_span("trace_analysis_example")
def analyze_model_performance(test_prompts: list, model: str = deployment_name):
    """
    Analyze model performance across multiple prompts with detailed tracing.
    """
    current_span = trace.get_current_span()
    current_span.set_attribute("analysis.prompt_count", len(test_prompts))
    current_span.set_attribute("analysis.model", model)
    
    results = []
    total_time = 0
    total_tokens = 0
    
    for i, prompt in enumerate(test_prompts):
        with tracer.start_as_current_span(f"test_prompt_{i+1}") as prompt_span:
            start_time = time.time()
            
            prompt_span.set_attribute("prompt.index", i + 1)
            prompt_span.set_attribute("prompt.content", prompt[:100])
            prompt_span.set_attribute("prompt.length", len(prompt))
            
            try:
                response = client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=150
                )
                
                elapsed_time = time.time() - start_time
                tokens_used = response.usage.total_tokens
                
                total_time += elapsed_time
                total_tokens += tokens_used
                
                prompt_span.set_attribute("response.time_seconds", elapsed_time)
                prompt_span.set_attribute("response.tokens", tokens_used)
                prompt_span.set_attribute("response.success", True)
                
                results.append({
                    "prompt": prompt,
                    "response": response.choices[0].message.content,
                    "time": elapsed_time,
                    "tokens": tokens_used,
                    "success": True
                })
                
            except Exception as e:
                elapsed_time = time.time() - start_time
                
                prompt_span.set_attribute("response.time_seconds", elapsed_time)
                prompt_span.set_attribute("response.success", False)
                prompt_span.set_attribute("error.type", type(e).__name__)
                prompt_span.record_exception(e)
                
                results.append({
                    "prompt": prompt,
                    "error": str(e),
                    "time": elapsed_time,
                    "success": False
                })
    
    # Calculate summary statistics
    successful_calls = [r for r in results if r["success"]]
    avg_time = total_time / len(test_prompts) if test_prompts else 0
    success_rate = len(successful_calls) / len(test_prompts) if test_prompts else 0
    
    current_span.set_attribute("analysis.total_time", total_time)
    current_span.set_attribute("analysis.average_time", avg_time)
    current_span.set_attribute("analysis.total_tokens", total_tokens)
    current_span.set_attribute("analysis.success_rate", success_rate)
    current_span.set_attribute("analysis.successful_calls", len(successful_calls))
    
    current_span.add_event(
        "analysis_completed",
        {
            "total_prompts": len(test_prompts),
            "successful_calls": len(successful_calls),
            "success_rate": success_rate,
            "total_time": total_time,
            "total_tokens": total_tokens
        }
    )
    
    return {
        "results": results,
        "summary": {
            "total_prompts": len(test_prompts),
            "successful_calls": len(successful_calls),
            "success_rate": success_rate,
            "total_time": total_time,
            "average_time": avg_time,
            "total_tokens": total_tokens
        }
    }

print("Model performance analysis function created!")

In [ ]:
# Test the performance analysis
test_prompts = [
    "What is Python?",
    "Explain machine learning in one sentence.",
    "What are the benefits of cloud computing?",
    "How does artificial intelligence work?"
]

try:
    analysis_result = analyze_model_performance(test_prompts)
    
    print("Performance Analysis Results:")
    print(f"Total prompts: {analysis_result['summary']['total_prompts']}")
    print(f"Successful calls: {analysis_result['summary']['successful_calls']}")
    print(f"Success rate: {analysis_result['summary']['success_rate']:.2%}")
    print(f"Total time: {analysis_result['summary']['total_time']:.2f} seconds")
    print(f"Average time per call: {analysis_result['summary']['average_time']:.2f} seconds")
    print(f"Total tokens used: {analysis_result['summary']['total_tokens']}")
    
except Exception as e:
    print(f"Error in performance analysis: {e}")

## 9. Error Handling and Debugging with Traces

Demonstrate how to use traces for debugging errors and monitoring application performance.

In [ ]:
@tracer.start_as_current_span("robust_ai_function")
def robust_ai_function(user_input: str, max_retries: int = 3, timeout: float = 30.0):
    """
    A robust AI function with comprehensive error handling and tracing.
    """
    current_span = trace.get_current_span()
    current_span.set_attribute("function.max_retries", max_retries)
    current_span.set_attribute("function.timeout", timeout)
    current_span.set_attribute("function.input_length", len(user_input))
    
    # Input validation
    if not user_input or len(user_input.strip()) == 0:
        error_msg = "Empty input provided"
        current_span.set_attribute("error.type", "ValidationError")
        current_span.set_attribute("error.message", error_msg)
        current_span.add_event("validation_failed", {"reason": "empty_input"})
        current_span.set_status(trace.Status(trace.StatusCode.ERROR, error_msg))
        raise ValueError(error_msg)
    
    if len(user_input) > 4000:
        error_msg = "Input too long (max 4000 characters)"
        current_span.set_attribute("error.type", "ValidationError")
        current_span.set_attribute("error.message", error_msg)
        current_span.add_event("validation_failed", {"reason": "input_too_long", "length": len(user_input)})
        current_span.set_status(trace.Status(trace.StatusCode.ERROR, error_msg))
        raise ValueError(error_msg)
    
    current_span.add_event("validation_passed")
    
    # Retry logic with tracing
    for attempt in range(max_retries):
        with tracer.start_as_current_span(f"attempt_{attempt + 1}") as attempt_span:
            attempt_span.set_attribute("retry.attempt_number", attempt + 1)
            attempt_span.set_attribute("retry.max_attempts", max_retries)
            
            try:
                start_time = time.time()
                
                # Simulate potential issues for demonstration
                if attempt == 0 and "error" in user_input.lower():
                    # Simulate a transient error on first attempt
                    attempt_span.add_event("simulated_transient_error")
                    raise Exception("Simulated transient network error")
                
                response = client.chat.completions.create(
                    model="gpt-4",
                    messages=[{"role": "user", "content": user_input}],
                    max_tokens=300,
                    timeout=timeout
                )
                
                elapsed_time = time.time() - start_time
                
                # Success - add attributes and return
                attempt_span.set_attribute("attempt.success", True)
                attempt_span.set_attribute("attempt.response_time", elapsed_time)
                attempt_span.set_attribute("attempt.tokens_used", response.usage.total_tokens)
                
                current_span.set_attribute("function.successful_attempt", attempt + 1)
                current_span.set_attribute("function.total_time", elapsed_time)
                current_span.set_attribute("function.success", True)
                
                current_span.add_event(
                    "function_completed_successfully",
                    {
                        "attempt": attempt + 1,
                        "response_time": elapsed_time,
                        "tokens_used": response.usage.total_tokens
                    }
                )
                
                return {
                    "content": response.choices[0].message.content,
                    "attempt": attempt + 1,
                    "response_time": elapsed_time,
                    "tokens_used": response.usage.total_tokens
                }
                
            except Exception as e:
                elapsed_time = time.time() - start_time
                
                attempt_span.set_attribute("attempt.success", False)
                attempt_span.set_attribute("attempt.error_type", type(e).__name__)
                attempt_span.set_attribute("attempt.error_message", str(e))
                attempt_span.set_attribute("attempt.time_to_failure", elapsed_time)
                
                attempt_span.add_event(
                    "attempt_failed",
                    {
                        "error_type": type(e).__name__,
                        "error_message": str(e),
                        "time_to_failure": elapsed_time
                    }
                )
                
                attempt_span.record_exception(e)
                
                if attempt == max_retries - 1:
                    # Final attempt failed
                    current_span.set_attribute("function.success", False)
                    current_span.set_attribute("function.final_error_type", type(e).__name__)
                    current_span.set_attribute("function.final_error_message", str(e))
                    current_span.set_attribute("function.attempts_exhausted", True)
                    
                    current_span.add_event(
                        "all_attempts_failed",
                        {
                            "total_attempts": max_retries,
                            "final_error": str(e)
                        }
                    )
                    
                    current_span.record_exception(e)
                    current_span.set_status(trace.Status(trace.StatusCode.ERROR, f"Failed after {max_retries} attempts: {str(e)}"))
                    raise
                else:
                    # Wait before retry
                    wait_time = 2 ** attempt  # Exponential backoff
                    attempt_span.add_event("waiting_before_retry", {"wait_seconds": wait_time})
                    time.sleep(wait_time)

print("Robust AI function with error handling and tracing created!")

In [ ]:
# Test successful case
print("Testing successful case:")
try:
    result = robust_ai_function("What are the main benefits of distributed tracing?")
    print(f"Success! Attempt: {result['attempt']}, Time: {result['response_time']:.2f}s")
    print(f"Response: {result['content'][:100]}...")
except Exception as e:
    print(f"Unexpected error: {e}")

print("\n" + "="*50 + "\n")

# Test retry scenario (simulated)
print("Testing retry scenario (contains 'error' to trigger simulation):")
try:
    result = robust_ai_function("Explain how error handling works in distributed systems.")
    print(f"Success after retries! Attempt: {result['attempt']}, Time: {result['response_time']:.2f}s")
    print(f"Response: {result['content'][:100]}...")
except Exception as e:
    print(f"Failed after all retries: {e}")

print("\n" + "="*50 + "\n")

# Test validation error
print("Testing validation error:")
try:
    result = robust_ai_function("")  # Empty input
    print(f"Unexpected success: {result}")
except ValueError as e:
    print(f"Expected validation error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

## Summary and Best Practices

This notebook demonstrated comprehensive tracing capabilities for AI applications using Azure AI Foundry. Here are the key takeaways:

### Key Features Demonstrated:
1. **Basic Tracing Setup** - Instrumentation and configuration
2. **Custom Spans** - Creating meaningful trace boundaries
3. **Attributes and Events** - Adding rich metadata to traces
4. **Error Handling** - Capturing and tracing errors effectively
5. **Performance Monitoring** - Tracking timing and resource usage
6. **Multi-step Workflows** - Tracing complex application flows

### Best Practices:
- Always instrument your OpenAI SDK calls
- Use meaningful span names that describe the operation
- Add relevant attributes for debugging and analysis
- Include error handling with proper exception recording
- Use events to mark important milestones
- Consider privacy when logging user inputs
- Monitor performance metrics like token usage and response times

### Next Steps:
- Set up dashboards in Application Insights
- Create alerts for performance thresholds
- Implement trace sampling for high-volume applications
- Use trace data for performance optimization
- Integrate with CI/CD pipelines for testing

In [ ]:
# Clean up and final notes
print("Notebook completed successfully!")
print("""\nTo view your traces:
1. Go to Azure AI Foundry portal (https://ai.azure.com)
2. Navigate to your project
3. Click on "Tracing" in the navigation menu
4. Explore the traces generated by this notebook

For production use:
- Configure appropriate sampling rates
- Set up proper authentication
- Monitor trace volume and costs
- Create custom dashboards and alerts
""")